In [1]:
import numpy as np
import os
import random
from matplotlib import pyplot as plt

class DataReader():
    def __init__(self):
        self.train_X, self.train_Y, self.test_X, self.test_Y = self.read_data()

    def read_data(self):
        filename = os.listdir("datasets")[0]
        file = open("datasets/" + filename)

        file.readline()

        data = []

        for line in file:
            splt = line.split(",")

            x, label = self.process_data(splt)

            data.append((x, label))

        random.shuffle(data)
        
        X = []
        Y = []

        for el in data:
            X.append(el[0])
            Y.append(el[1])

        X = np.asarray(X)
        Y = np.asarray(Y)

        train_X = X[:int(len(X) * 0.8)]
        train_Y = Y[:int(len(Y) * 0.8)]
        test_X = X[int(len(X) * 0.8):]
        test_Y = Y[int(len(Y) * 0.8):]

        return train_X, train_Y, test_X, test_Y

    def process_data(self, splt):
        data = []
        for i in range(1, len(splt)):
            if splt[i] == "": splt[i] = 0
        data = list(map(float, splt[1:-1]))

        label = list(map(int, splt[-1:]))

        return data, label


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


class MLPC():
    def init(self):
        self.model = self.new_model()

    def new_model(self,hp):
        model = keras.Sequential()
        model.add(keras.layers.Dense(11))
        for i in range(hp.Int('numlayers', 2, 5)):
            model.add(keras.layers.Dense(units=hp.Int('units' + str(i),
                                                min_value=16,
                                                maxvalue=180),
                                   activation='relu'))
            model.add(keras.layers.BatchNomaliation())
            model.add(keras.layers.Dropout(hp.Float('dropout' + str(i), min_value=0.0, max_value=0.5, step=0.05)))
        model.add(keras.layers.Dense(7, activation='softmax'))

        hp_learning_rate = hp.Float('learning_rate', min_value=1e-6, max_value=1e-3, sampling='LOG')
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate), loss='categorical_crossentropy',
                      metrics=['accuracy'])
        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Int('batch_size', min_value = 16, max_value = 256, step = 16),
            **kwargs,
        )


In [6]:
from tensorflow import keras

# import DataReader
# import MLPC
import matplotlib.pyplot as plt
import pandas as pd

EPOCHS = 200

dr = DataReader()

# mlp = MLPC().model

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = MLPC().fit()
# history = mlp.fit(dr.train_X, dr.train_Y, epochs=EPOCHS, batch_size=512,
#         validation_data=(dr.test_X, dr.test_Y),
#         callbacks=[early_stop])

print()

history_df = pd.DataFrame(history.history)
history_df['loss'].plot()
history_df['val_loss'].plot()
plt.legend()
plt.show()

history_df = pd.DataFrame(history.history)
history_df['accuracy'].plot()
history_df['val_accuracy'].plot()
plt.legend()
plt.show()

AttributeError: 'MLPC' object has no attribute 'history'